In [1]:
# Change directory to workspace root so that relative path loads work correctly.
from IPython.display import Markdown as md
import datetime
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

from ia.common.jira.connection import connect
import ia.quality.conf.components as components
import ia.quality.algo as algo
import ia.common.viz.conf.dashboard as confboard


report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

/Users/lswolkien/Devel/EngineeringMetrics


`Report executed at 2020-07-08 17:39`

# Report variables

In [2]:
conf_space_key = "~lswolkien"
conf_parent_page = "Reports"
conf_page_title = "Quality metrics"
PROJECT_KEYS = ('DANSRE', 'DANMR', 'DANCOE', 'DANSDP', 'DANTF', 'DANMIG', 'DANUNCO', 'DANMRGCORE')

STATUS_DONE=('Done')
ISSUE_TYPES=('Incident')#, 'Bug', 'User Story Bug')

END_DATE = "2020-07-01"
SPRINT_LENGTH = 14
NO_SPRINTS = 5

report_title = f'Quality metrics dashboard'
report_description = f"Quality Report: {report_start_time:%Y-%m-%d}"



# Access variables

In [3]:
jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

## Calculate metrics based on Jira data

In [4]:
jira = connect(jira_url, basic_auth=(jira_username, jira_password))
dashboard = confboard.Dashboard(
    conf_url, 
    conf_username, 
    conf_password, 
    conf_page_title, 
    conf_space_key, 
    conf_parent_page
)


all_active_maintenance = algo.maintenance_backlog(jira, PROJECT_KEYS, ISSUE_TYPES, STATUS_DONE)
maintenance_backlog_issue_keys = [i.key for i in all_active_maintenance]
periods = algo.get_periods(END_DATE, SPRINT_LENGTH, NO_SPRINTS)

trend = algo.maintenance_trend(
    jira, 
    periods,
    PROJECT_KEYS, 
    ISSUE_TYPES
)

report_head = confboard.Component(confboard.report_head, [report_title, report_description])
# labels = [f'Sprint {i}' for i in range(28, 23, -1)]
end_dates = [i[1] for i in periods]
labels = [f'Sprint {i}({d:%m-%d})' for i, d in enumerate(end_dates, 24)]
quality_report = confboard.Component(components.maintenance_report, [labels, trend, maintenance_backlog_issue_keys, "DM channel"])

dashboard.publish([report_head, quality_report])

In [5]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)

` Report finished at 2020-07-08 17:39, time elapsed: 0:00:20.208178`

In [6]:
import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)